In [ ]:
#Step 1.1- #Loading the saved_model
import tensorflow as tf
import time
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL="pre-trained-models/my_model_640x640/saved_model"
#PATH_TO_SAVED_MODEL="/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/Sketch2Code_model_v2/saved_model"

print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')

In [ ]:
#Step 1.2- #Setting Threshold Vaalue
THRESHOLD_VALUE = 0.4
print('Threshold value: '+ str(THRESHOLD_VALUE))
KEY_BOXES = 'detection_boxes'
KEY_CLASSES= 'detection_classes'
KEY_SCORES = 'detection_scores'
KEY_CLASS_NAMES='detection_class_names'
KEY_BOX_AREAS='detection_box_areas'
ROOT_ID = "root"
APPBAR_ID = "appBar"
BOTTOMNAV_ID = "bottomNavigation"
CONTENT_ID = "layoutContent"

In [ ]:
#Step 2- #Loading the label_map
from object_detection.utils import label_map_util
class_by_no = label_map_util.create_category_index_from_labelmap(
    "annotations/label_map.pbtxt", use_display_name=True
)

class_by_name = dict()


def create_class_by_name():
    for value in class_by_no.values():
        name = value["name"]
        class_by_name[name] = value


def get_name_for_class(class_no):
    return class_by_no[class_no]["name"]


def get_class_for_name(name):
    return class_by_name[name]["id"]

create_class_by_name()




#category_index=label_map_util.create_category_index_from_labelmap([path_to_label_map],use_display_name=True)

In [ ]:
#Step 3- Getting Detections Dict from Vision Model

import numpy as np
from PIL import Image
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))



# infer single image and return detection dictionary

def infer_single_img(image_np):
    print('Running inference for img ....')
    
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor=input_tensor[tf.newaxis, ...]
    
    # Get Detections for Tensor
    detections=detect_fn(input_tensor)
    
    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}
    detections['num_detections']=num_detections
    
    # detection_classes should be ints.
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)
    
    print('Done!')
    return detections
    
    


def show_virtualization(detections,image_np_with_detections):
    # Virtulaize in matlab
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections[KEY_BOXES],
          detections[KEY_CLASSES],
          detections[KEY_SCORES],
          class_by_no,
          use_normalized_coordinates=True,
          max_boxes_to_draw=50,     #max number of bounding boxes in the image
          min_score_thresh= THRESHOLD_VALUE, #min prediction threshold
          agnostic_mode=False)
    %matplotlib inline
    mpl.rcParams['figure.dpi']= 250
    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
    plt.show()
    

    #Infer single image from Image numpy array and Virtualize
def infer_and_show_virtualization(image_np):
    # get Image detections dict
    detections = infer_single_img(image_np)
    
     # Copy Image numpy array
    image_np_with_detections=image_np.copy()
    
    #Virtualize the detections
    show_virtualization(detections,image_np_with_detections)
    

    
from scipy import ndimage

def rotate_box_rightangle_clockwise(box):
    ymin, xmin, ymax, xmax = box
    
    #Rotate box
    yrmin= xmin
    xrmin= 1-ymax
    yrmax= xmax
    xrmax= 1-ymin
    
    return [yrmin, xrmin, yrmax, xrmax]


def rotate_image(image,degree):
    return ndimage.rotate(image,degree)

def rotate_detections(detections):
    boxes = detections[KEY_BOXES]
    rotated_list =  [rotate_box_rightangle_clockwise(box) for box in boxes]
    detections[KEY_BOXES] = np.array(rotated_list)
    
def filter_by_score(detections):
    # getting filter array according to the Treshold
    threshold_filter = detections[KEY_SCORES] >= THRESHOLD_VALUE
    
    # Filterinf SCORES,CLASSES,BOXES according to the filter
    filtered_detections = dict()
    filtered_detections[KEY_SCORES] = detections[KEY_SCORES][threshold_filter]
    filtered_detections[KEY_CLASSES] = detections[KEY_CLASSES][threshold_filter]
    filtered_detections[KEY_BOXES] = detections[KEY_BOXES][threshold_filter]
    
    return filtered_detections

def filter_by_class(detections):
    # getting filter array according to the Treshold
    threshold_filter = detections[KEY_SCORES] >= THRESHOLD_VALUE
    
    # Filterinf SCORES,CLASSES,BOXES according to the filter
    filtered_detections = dict()
    filtered_detections[KEY_SCORES] = detections[KEY_SCORES][threshold_filter]
    filtered_detections[KEY_CLASSES] = detections[KEY_CLASSES][threshold_filter]
    filtered_detections[KEY_BOXES] = detections[KEY_BOXES][threshold_filter]
    
    return filtered_detections
    

In [ ]:
#Step 3.2- #Loading the single image
imagePath = 'Dataset/Labeled/Dataset2-640x640/img2.jpg'
print(imagePath)
# Load image as np array
image_np = load_image_into_numpy_array(imagePath)

In [ ]:
#Step 4- Assign Global detection for testing 
detections = infer_single_img(image_np)

In [ ]:
#Step 5 - Virtualize Detections
image = rotate_image(image_np,-90)
detections_by_score = filter_by_score(detections)
rotate_detections(detections_by_score)
show_virtualization(detections_by_score,image.copy())

In [ ]:
# Step 6- Import Algorithms
import GroupingAlgo 
import OverlapAlgo 
import CodeGen
import LevelAlgo

In [ ]:
# Step 7 - Create Empty dictionary and Start Resolving and Grouping
ui_dict= dict()
GroupingAlgo.group_and_resolve(detections_by_score,ui_dict,class_by_no)

In [ ]:
# Step 8 - Genarate Code
root_code = CodeGen.constraint_layout_code(ui_dict[CONTENT_ID])

In [ ]:
print(root_code)